In [19]:
import jq
from vespa.application import Vespa
from vespa.io import VespaQueryResponse

# document_set = "8th EAP 2023 progress report"
# document_id = "https://water.europa.eu/freshwater/europe-freshwater/water-framework-directive/surface-water-chemical-status/priority-substances-causing-failure-to-good-chemical-status/de-3rd-cybutryne"

url = "http://10.50.5.60:64797/"
app = Vespa(url=url)

document_ids = None

with open('document_ids.json') as f:
    document_ids = json.load(f)

print(len(document_ids))

26008


In [36]:
import os

def download_document(document_id, session):
    query = f"""
        select *
        from sources *
        where document_id in ("{document_id}")
        order by chunk_id
        limit 400
    """
    try:
        response = session.query(yql=query)
    except:
        print(f"Error in {document_id}")
        return
    assert response.is_successful()
    data = response.get_json()
    
    for chunk in data['root'].get('children', []):
        for document_set in chunk['fields'].get('document_sets', ['no-document-set']):
            parent = os.path.join('data-download', document_set)
            if not os.path.exists(parent):
                os.makedirs(parent)
            fname = f"{chunk['id']}.json"
            with open(os.path.join(parent, fname), 'w') as f:
                json.dump(chunk, f)
                # print(f"Downloaded {chunk['id']}")

In [37]:
from tqdm.notebook import tqdm

for i in tqdm(range(len(document_ids)), desc="Saving documents"):
    with app.syncio() as session:
        doc_id = document_ids[i]
        download_document(doc_id, session)

Saving documents:   0%|          | 0/26008 [00:00<?, ?it/s]

Error in FILE_CONNECTOR__7c2a33e6-b2f8-4234-81e5-1613893d5b6a/Puppet\Foreman.txt
Error in FILE_CONNECTOR__7c2a33e6-b2f8-4234-81e5-1613893d5b6a/Setup_of_the_staff_noticeboard_using_the_"News"_functionality.txt
Error in FILE_CONNECTOR__7c2a33e6-b2f8-4234-81e5-1613893d5b6a/As_an_Admin_push_"Create_Permissions"_button.txt
Error in FILE_CONNECTOR__7c2a33e6-b2f8-4234-81e5-1613893d5b6a/Admin_push_"Create_Permissions"_button.txt
